# Семинар 5    
## Собираем поисковик 

![](https://bilimfili.com/wp-content/uploads/2017/06/bir-urune-emek-vermek-o-urune-olan-deger-algimizi-degistirir-mi-bilimfilicom.jpg) 


Мы уже все знаем, для того чтобы сделать поисковик. Осталось соединить все части вместе.    
Итак, для поисковика нам понадобятся:         
**1. База документов **
> в первом дз - корпус Друзей    
в сегодняшнем дз - корпус юридических вопросов-ответов    
в итоговом проекте - корпус Авито   

**2. Функция индексации**                 
Что делает: собирает информацию о корпусе, по которуму будет происходить поиск      
Своя для каждого поискового метода:       
> A. для обратного индекса она создает обратный индекс (чудо) и сохраняет статистики корпуса, необходимые для Okapi BM25 (средняя длина документа в коллекции, количество доков ... )             
> B. для поиска через word2vec эта функция создает вектор для каждого документа в коллекции путем, например, усреднения всех векторов коллекции       
> C. для поиска через doc2vec эта функция создает вектор для каждого документа               

   Не забывайте сохранить все, что насчитает эта функция. Если это будет происходить налету во время поиска, понятно, что он будет работать сто лет     
   
**3. Функция поиска**     
Можно разделить на две части:
1. функция вычисления близости между запросом и документом    
> 1. для индекса это Okapi BM25
> 2. для w2v и d2v это обычная косинусная близость между векторами          
2. ранжирование (или просто сортировка)


Время все это реализовать.

## В предыдущей серии

In [18]:
import os
import json
import string
import pickle

from collections import defaultdict, Counter
from string import punctuation, digits
from math import log

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize

from judicial_splitter import splitter
from tqdm import tqdm_notebook
from pymystem3 import Mystem


punctuation = set(punctuation + '«»—–…“”\n\t' + digits)
mystem = Mystem()


def preprocess_words(mystem, text):
    table = str.maketrans({ch: ' ' for ch in punctuation})
    
    tokenized = word_tokenize(text.replace('\ufeff', '').lower().translate(table))
    return [mystem.lemmatize(word)[0] for word in tokenized], len(tokenized)


def preprocess_files(mystem, file, files_list):
    with open(file, 'r', encoding='utf-8') as f:
        words_list, text_length = preprocess_words(mystem, f.read())
        document_length[files_list.index(file)] = text_length
        
        return words_list
    
def get_inverted_index(mystem, files_list, save=True):

    inverted_index = defaultdict(list)
    global document_length
    document_length = [None] * len(files_list)
    for file in files_list:
        for word in preprocess_files(mystem, file, files_list):
            inverted_index[word].append(files_list.index(file))
            
    if save:
        with open('inverted_index.json', 'w', encoding='utf-8') as fw:
            json.dump(inverted_index, fw, ensure_ascii=False)     
        with open('document_length.json', 'w', encoding='utf-8') as fw:
            json.dump(document_length, fw, ensure_ascii=False)
    
    return inverted_index, document_length


def score_BM25(input_, episod, avgdl, k1, b, N, n, data):
    arr = mystem.lemmatize(cleaner(input_))
    result = 0 
    for word in arr:
        if (word in data.index) and (episod in data.columns):
            n = df_inv.loc[word]['count'] 
            f = data.loc[word][episod] / (log((N - n + 0.5) / n + 0.5))
            a = (k1 + 1) * f
            b = f + k1 * (1 - b + b * (len(arr)) / avgdl)
            result += (a / b) * log((N - n + 0.5) / n + 0.5)
    


def compute_sim(text, doc, data):

    relevance_score = score_BM25(text, doc, avgdl, k1, b, N, n, data)
    
    return relevance_score



def get_search_result(query, inverted_index, mystem, files_list, document_length, num_res):

    relevance_dict = defaultdict(float)
    lemmas, _ = preprocess_words(mystem, query)
    
    for lemma in lemmas:
        score = compute_sim(lemma, inverted_index, document_length)
        for elem in score:
            relevance_dict[elem] += score[elem]
            
    result = sorted(relevance_dict, key=relevance_dict.get, reverse=True)[:num_res]
    
    return [files_list[ind] for ind in result]

# Индексация

In [19]:
from gensim.models import Word2Vec, KeyedVectors
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.models.fasttext import FastText

In [20]:
from nltk.corpus import stopwords
russian_stopwords = set(stopwords.words('russian'))

## Word2Vec
### Задание 1
Загрузите любую понравившуюся вам word2vec модель

In [21]:
w2v_model = FastText.load('/Users/alinashaymardanova/Downloads/araneum_none_fasttextskipgram_300_5_2018/araneum_none_fasttextskipgram_300_5_2018.model')
w2v_model.init_sims(replace=True)

### Задание 2 
Напишите функцию индексации для поиска через word2vec. Она должна для каждого документа из корпуса строить вектор.   
Все вектора надо сохранить, по формату советую json. При сохранении не забывайте, что вам надо сохранить не только  вектор, но и опознователь текста, которому он принадлежит. 
Для поисковика это может быть url страницы, для поиска по текстовому корпусу сам текст.

> В качестве документа для word2vec берите **параграфы** исходного текста, а не весь текст целиком. Так вектора будут более осмысленными. В противном случае можно получить один очень общий вектор, релевантый совершенно разным запросам.

In [22]:
mystem = Mystem()

In [23]:
def preprocessing(mystem, input_text, del_stopwords=True, del_digit=False):

    russian_stopwords = set(stopwords.words('russian'))
    words = [x.lower().strip(string.punctuation + '»«–…—') for x in nltk.word_tokenize(input_text)]
    lemmas = [mystem.lemmatize(x)[0] for x in words if x]
    
    lemmas_arr = []
    for lemma in lemmas:
        if del_stopwords:
            if lemma in russian_stopwords:
                continue
        if del_digit:
            if lemma.isdigit():
                continue
        
        lemmas_arr.append(lemma)
    return lemmas_arr

In [24]:
def get_w2v_vectors(model, lemmas):
    """Получает вектор документа"""
    
    vectors_ = [] 
    for word in lemmas:
        try:
            vectors_.append(model.wv[word])
        except:
            continue
            
    return (sum(vectors_)/len(vectors_))


def save_w2v_base(files, model, mystem, save=True):
    """Индексирует всю базу для поиска через word2vec"""
    
    info_ = []    
    
    for file in tqdm_notebook(files):
        with open(file, 'r', encoding='utf-8') as f:
            text = f.read()
            lemmas = preprocessing(mystem, text)
            vec = get_w2v_vectors(model, lemmas)
            info_.append({'file': file, 'word2vec': vec})
    
    if save:
        with open('w2v_base.pkl', 'wb') as fw:
            pickle.dump(info_, fw)
    
    return info_

In [25]:
path_to_files = '/Users/alinashaymardanova/Downloads/article/'
files = list(map(lambda x: path_to_files + x, os.listdir(path_to_files)[:1000]))

In [26]:
files[0]

'/Users/alinashaymardanova/Downloads/article/220648.txt'

In [27]:
data_word2vec = save_w2v_base(files, w2v_model, mystem)

A Jupyter Widget

## Doc2Vec
### Задание 3
Напишите функцию обучения doc2vec на юридических текстах, и получите свою кастомную d2v модель. 
> Совет: есть мнение, что для обучения doc2vec модели не нужно удалять стоп-слова из корпуса. Они являются важными семантическими элементами.      

Важно! В качестве документа для doc2vec берите **параграфы** исходного текста, а не весь текст целиком. И не забывайте про предобработку.

In [34]:
def paragraph(files_list, mystem):
    text_ = {}
    data = []
    
    for file in files:
        with open(file, 'r', encoding='utf-8') as f:
            text = f.read()
            text_[file] = text
            paragraphs = splitter(text, 1)
            for el in paragraphs:
                paragraph_lemmatized = preprocessing(mystem, el, del_stopwords=False)
                data.append({'file': file, 'paragraph': paragraph_lemmatized})
    
    with open('file_text', 'w', encoding='utf-8') as fw:
        json.dump(text_, fw)
    
    return data, text_

In [35]:
def train_doc2vec(data, epochs):
    tagged_data = [TaggedDocument(words=elem['paragraph'],tags=[str(i)]) for i, elem in enumerate(data)]
    model = Doc2Vec(vector_size=100, min_count=5, alpha=0.025, min_alpha=0.025, epochs=epochs, workers=4, dm=1)
    
    model.build_vocab(tagged_data)
    model.train(tagged_data, total_examples=model.corpus_count, epochs=model.epochs)
    
    return model

In [37]:
paragraphs, file_text = paragraph(files, mystem)

In [39]:
d2v_model = train_doc2vec(paragraphs, 500)

4957
CPU times: user 17min 59s, sys: 3min 22s, total: 21min 21s
Wall time: 14min 45s


### Задание 4
Напишите функцию индексации для поиска через doc2vec. Она должна для каждого документа из корпуса получать вектор.    
Все вектора надо сохранить, по формату советую json. При сохранении не забывайте, что вам надо сохранить не только вектор, но и опознователь текста, которому он принадлежит. 

In [40]:
def get_d2v_vectors(model, lemmas):
    """Получает вектор документа"""
    vec = model.infer_vector(lemmas)
    return vec
    
def save_d2v_base(model, paragraphs, save=True):
    """Индексирует всю базу для поиска через doc2vec"""
    documents_info = []    
    
    for paragraph in paragraphs:
        vec = get_d2v_vectors(model, paragraph['paragraph'])
            
        file_info = {'file': paragraph['file'], 'doc2vec': vec}
        documents_info.append(file_info)
    
    if save:
        with open('d2v_base.pickle', 'wb') as fw:
            pickle.dump(documents_info, fw)
    
    return documents_info

In [41]:
%%time

data_doc2vec = save_d2v_base(d2v_model, paragraphs)

CPU times: user 8min 57s, sys: 5.59 s, total: 9min 3s
Wall time: 9min 21s


# Функция поиска

Для обратного индекса функцией поиска является Okapi BM25. Она у вас уже должна быть реализована.

Функция измерения близости между векторами нам пригодится:

In [76]:
from gensim import matutils
import numpy as np 

def similarity(v1, v2):
    v1_norm = matutils.unitvec(np.array(v1))
    v2_norm = matutils.unitvec(np.array(v2))
    return np.dot(v1_norm, v2_norm)

### Задание 5
Напишите функцию для поиска через word2vec и для поиска через doc2vec, которая по входящему запросу выдает отсортированную выдачу документов.

In [81]:
def search_inv(query, questions, answers, inv_index) -> list:
    """
    Search documents relative to query using inverted index algorithm.
    :param query: str: input text
    :param questions: list: all questions from corpus
    :param answers: list: all answers from corpus
    :param inv_index: list: questions inverted index
    :return: list: 5 relative answers
    """
    k1 = 2.0
    b = 0.75
    avgdl = np.mean(list(map(len, questions)))
    N = len(questions)
    
    query_list = preprocessing(query)
    scores = list()
    
    for i, doc in enumerate(questions):
        score = 0
        for word in query_list:
            score += compute_sim(word, doc, inv_index, k1, b, avgdl, N)
        scores.append([i, score])
        
    ranked = sorted(scores, key = lambda x: x[1], reverse=True)
    result = [{'id': doc[0], 'text': answers[doc[0]]} for doc in ranked[:5]]

    return result

def search_w2v(query, w2v_base_quest, answers) -> list:
    """
    Search documents relative to query using inverted w2v algorithm.
    :param query: str: input text
    :param w2v_base_quest: list: all questions' vectors from corpus
    :param answers: list: all answers from corpus
    :return: list: 5 relative answers
    """
    
    similarities = list()

    for part in sp(query, 3):
        lemmas = preprocessing(query)
        vec = get_w2v_vectors(lemmas)
    
        for quest in w2v_base_quest:
            s = similarity(vec, quest['vec'])
            similarities.append({'id': quest['id'], 'sim': s})

    ranked = sorted(similarities, key=lambda x: x['sim'], reverse=True)
    result = [{'id': doc['id'], 'text': answers[doc['id']]} for doc in ranked[:5]]
    
    return result
    
def search_d2v(query, d2v_base_quest, answers) -> list:
    """
    Search documents relative to query using inverted d2v algorithm.
    :param query: str: input text
    :param d2v_base_quest: list: all questions' vectors from corpus
    :param answers: list: all answers from corpus
    :return: list: 5 relative answers
    """
    similarities = list()

    for part in sp(query, 3):
        lemmas = preprocessing(query)
        vec = get_d2v_vectors(lemmas)
    
        for quest in d2v_base_quest:
            s = similarity(vec, quest['vect'])
            similarities.append({'id': quest['id'], 'sim': s})

    ranked = sorted(similarities, key=lambda x: x['sim'], reverse=True)
    result = [{'id': doc['id'], 'text': answers[doc['id']]} for doc in ranked[:5]]   
    
    return result

In [86]:
%%time

inverted_index, document_length = get_inverted_index(mystem, files_list)

CPU times: user 55.2 s, sys: 12.1 s, total: 1min 7s
Wall time: 2min 25s


In [91]:
with open('inverted_index.json', 'r', encoding='utf-8') as f:
    inverted_index = json.load(f)

In [92]:
with open('document_length.json', 'r', encoding='utf-8') as f:
    document_length = json.load(f)

После выполнения всех этих заданий ваш поисковик готов, поздравляю!                  
Осталось завернуть все написанное в питон скрипт, и сделать общую функцию поиска гибким, чтобы мы могли искать как по обратному индексу, так и по word2vec, так и по doc2vec.          
Сделать это можно очень просто через старый добрый ``` if ```, который будет дергать ту или иную функцию поиска:

In [51]:
def search(query, search_method, n_results=5):
    
    if search_method == 'inverted_index':
        search_result = get_search_result(query, inverted_index, mystem, files_list, document_length, n_results)
    
    elif search_method == 'word2vec':
        preprocessed_query = preprocessing(mystem, query)
        search_result = search_w2v(preprocessed_query, w2v_model, data_word2vec, n_results)
    
    elif search_method == 'doc2vec':
        preprocessed_query = preprocessing(mystem, query, del_stopwords=False)
        search_result = search_d2v(preprocessed_query, d2v_model, data_doc2vec, n_results)
    
    else:
        raise TypeError('unsupported search method')
    
    results = [(filename, file_text[filename]) for filename in search_result]
    return results

In [52]:
search('Высшего Арбитражного Суда Российской Федерации', 'doc2vec')

NameError: name 'search_d2v' is not defined